In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('../NLP/fake-news/train.csv', error_bad_lines=False)

In [3]:
df.dropna(axis=0, how='any', inplace=True)

In [4]:
y=df['label']

In [5]:
X=df.drop('label',axis=1)

In [55]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [8]:
messages=X.copy()

In [9]:
messages.drop(['id','author','text'],axis=1, inplace=True)

In [11]:
messages.reset_index(inplace=True)

In [12]:
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
ps = PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
    review=re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [13]:
from tensorflow.keras.preprocessing.text import one_hot

In [66]:
voc_size=20000

In [67]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[5333, 9317, 14104, 1821, 2288, 1963, 12317, 2996, 4358, 10425],
 [10196, 15992, 15662, 1385, 5656, 918, 13521],
 [15914, 638, 13806, 17631],
 [15842, 12129, 12258, 12823, 11186, 11173],
 [4678, 5656, 13094, 18186, 10987, 19563, 5656, 785, 763, 1362],
 [2587,
  4305,
  14636,
  12707,
  12422,
  16446,
  13531,
  9189,
  287,
  11798,
  2857,
  10347,
  9583,
  6364,
  13521],
 [7998, 9945, 7364, 16394, 6411, 15395, 16384, 7102, 17803, 10059, 14557],
 [11343, 7373, 19659, 8701, 3099, 2101, 16446, 14059, 17803, 10059, 14557],
 [7489, 14063, 10278, 13356, 15965, 17068, 16593, 9033, 16446, 8244],
 [10083, 2827, 10184, 13505, 12884, 17823, 1396, 18480],
 [10980, 17291, 6028, 1238, 12449, 9844, 4182, 14613, 7969, 19137, 15681],
 [12823, 18375, 2288, 17068, 16446, 3099],
 [316, 9044, 15717, 6903, 8349, 10243, 18053, 7451, 13027],
 [988, 6911, 7662, 15899, 10457, 14570, 6423, 17803, 10059, 14557],
 [18595, 7250, 6288, 10796, 2875, 17803, 10059, 14557],
 [5710, 16957, 13501, 11092, 14171, 190

In [68]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ...  2996  4358 10425]
 [    0     0     0 ...  5656   918 13521]
 [    0     0     0 ...   638 13806 17631]
 ...
 [    0     0     0 ... 17803 10059 14557]
 [    0     0     0 ...  9682 19446 11935]
 [    0     0     0 ... 12393  7367 13820]]


In [69]:
embedding_vector_features=40 #size of each output vector will be 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            800000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 913,001
Trainable params: 913,001
Non-trainable params: 0
_________________________________________________________________
None


In [70]:
from sklearn.model_selection import train_test_split
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [71]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [72]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=10, batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 10s 794us/sample - loss: 0.3075 - acc: 0.8518 - val_loss: 0.1946 - val_acc: 0.9178
Epoch 2/10
12250/12250 [==============================] - 9s 723us/sample - loss: 0.1156 - acc: 0.9576 - val_loss: 0.1965 - val_acc: 0.9208
Epoch 3/10
12250/12250 [==============================] - 9s 739us/sample - loss: 0.0538 - acc: 0.9815 - val_loss: 0.2810 - val_acc: 0.9203
Epoch 4/10
12250/12250 [==============================] - 10s 822us/sample - loss: 0.0229 - acc: 0.9939 - val_loss: 0.3708 - val_acc: 0.9115
Epoch 5/10
12250/12250 [==============================] - 10s 826us/sample - loss: 0.0164 - acc: 0.9949 - val_loss: 0.4138 - val_acc: 0.9127
Epoch 6/10
12250/12250 [==============================] - 11s 859us/sample - loss: 0.0071 - acc: 0.9980 - val_loss: 0.4623 - val_acc: 0.9107
Epoch 7/10
12250/12250 [==============================] - 10s 835us/sample - loss: 0.0045 - acc: 0.9987 - v

In [73]:
#Performance Metrics and Accuracy

In [74]:
y_pred=model.predict(X_test)

In [75]:
y_pred=np.where(y_pred > 0.5,1,0)

In [76]:
from sklearn.metrics import accuracy_score #Aoc Roc curve
accuracy_score(y_test,y_pred)

0.9148301574150787

In [77]:
from sklearn.metrics import classification_report, confusion_matrix

In [78]:
confusion_matrix(y_test,y_pred)

array([[3091,  328],
       [ 186, 2430]], dtype=int64)